# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to Moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model
**Q1**: Expand the above definition of $ p(\vec{w})$ using naive estimates of the parameters, such as $  p(w_4 \mid w_2, w_3) \stackrel{\tiny{\mbox{def}}}{=}  \frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} $ where \( C(w_2 w_3 w_4) \) denotes the count of times the trigram $ w_2 w_3 w_4 $ was observed in a training corpus.

**Write your answer:**

\begin{align}
p(\vec{w}) &= \frac{C(w_1)}{C(W)} \cdot \frac{C(w_1w_2)}{C(w_1)} \cdot \frac{C(w_1w_2w_3)}{C(w_1w_2)} \cdot \frac{C(w_2w_3w_4)}{C(w_2w_3)} \dots \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})} \\
&= \frac{1}{C(W)} \cdot \frac{1}{1} \cdot \frac{C{w_1w_2w_3}}{1} \cdot \frac{C(w_2w_3w_4)}{C(w_2w_3)} \dots \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})} \\
&= \frac{C{w_1w_2w_3}}{C(W)} \cdot \frac{C(w_2w_3w_4)}{C(w_2w_3)} \dots \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})} \\
\end{align}



**Q2**: One could also define a kind of reversed trigram language model $p_{reversed}$ that instead assumed the words were generated in reverse order (from right to left):
\begin{align} p_{reversed}(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}&p(w_n) \cdot p(w_{n-1} \mid w_n) \cdot p(w_{n-2} \mid w_{n-1} w_n) \cdot p(w_{n-3} \mid w_{n-2} w_{n-1}) \\ &\cdots p(w_2 \mid w_3 w_4) \cdot p(w_1 \mid w_2 w_3) \end{align}
By manipulating the notation, show that the two models are identical, i.e., $ p(\vec{w}) = p_{reversed}(\vec{w}) $ for any $ \vec{w} $ provided that both models use MLE parameters estimated from the same training data (see Q1 above).

**Write your answer:**

\begin{align}
p_{reversed}(\vec{w}) &=\frac{C(w_1w_2w_3)}{C(w_2w_3)} \cdot \frac{C(w_2w_3w_4)}{C(w_3w_4)} \dots \frac{C(w_{n-3}w_{n-2}w_{n-1})}{C(w_{n-2}w_{n-1})} \cdot \frac{C(w_{n-2}w_{n-1}w)}{C({w_{n-1}w_n})} \cdot \frac{C(w_{n-1}w_n)}{C(w_n)} \cdot \frac{C(w_n)}{C(W)} \\
&= \frac{C(w_1w_2w_3)}{1} \cdot \frac{C(w_2w_3w_4)}{C(w_2w_3)} \cdot \frac{C(w_3w_4w_5)}{C(w_3w_4)} \dots \frac{C(w_{n-2}w_{n-1}w)}{C(w_{n-2}w_{n-1})} \cdot \frac{C(w_{n-1}w_n)}{C(w_{n-1}w_n)} \cdot \frac{C(w_n)}{C(w_n)} \cdot \frac{1}{C(W)} \\
&= \frac{C(w_1w_2w_3)}{C(W)} \cdot \frac{C(w_2w_3w_4)}{C(w_2w_3)} \cdot \frac{C(w_3w_4w_5)}{C(w_3w_4)} \dots \frac{C(w_{n-2}w_{n-1}w)}{C(w_{n-2}w_{n-1})} \\
\end{align}

Now, with some simplification, we can show that the formula of $p_{reversed}(\vec{w})$ is the same as the one of $p(\vec{w})$ answered in the previous question.


## 2 $N$-gram Language Model Implementation

In [2]:
!wget -O train.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/test.txt

--2023-10-23 00:37:14--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6640478 (6.3M) [text/plain]
Saving to: 'train.txt'

train.txt           100%[===================>]   6.33M  12.6MB/s    in 0.5s    

2023-10-23 00:37:15 (12.6 MB/s) - 'train.txt' saved [6640478/6640478]

--2023-10-23 00:37:15--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Le

### 2.1 Building vocabulary

In [3]:
import math
from itertools import product

import numpy as np

from copy import deepcopy

import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /Users/hao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Global Variables

# threshold of frequency to be classified as <UNK>
G_FREQ_THRE = 3
G_UNK = "<UNK>"
G_SOS = "<s>"
G_EOS = "</s>"

In [5]:
# PARAMETERS
G_N = 1
G_WORD_TOKENS = []
G_VOCAB_TABLE = {}

In [6]:
with open("train.txt", 'r') as trainFile:
    # sentences = ["<s> " + line.strip() + " </s>" for line in trainFile]
    trainData = [line.strip() for line in trainFile]
    
with open("dev.txt", 'r') as devFile:
    devData = [line.strip() for line in devFile]
    
with open("test.txt", 'r') as testFile:
    testData = [line.strip() for line in testFile]

Import the training datset, and get sentence tokenization out of the training dataset, also add starting/end symbols for each sentence.

In [7]:
def getCustomWordTokens(sentence):
    pattern = r'<\w+>|[#$"\(\)\',.!-]|\d+(?:,\d{3})*(?:\.\d+)?|\w+(?:-\w+)*|\S+'
    G_VOCAB_TABLE
    tokenizer = RegexpTokenizer(pattern)
    wordTokens = tokenizer.tokenize(sentence)
    return wordTokens

def getNLTKWordTokens(sentence):
    wordTokens = word_tokenize(sentence)
    return wordTokens

def tokenize(sentences):
    word_tokens = []
    for sentence in sentences:
        # add SOS
        result = [G_SOS] * max(1, (G_N - 1))
        # tokenization
        tokens = getCustomWordTokens(sentence)
        # concatenate tokens and EOS
        result.extend(tokens)
        result.append(G_EOS)
        # formulation
        word_tokens.extend(result)
    return word_tokens
        
def initLM(n = 1):
    global G_TOKENIZED_SENTS, G_WORD_TOKENS, G_VOCAB_TABLE, G_N 
    G_TOKENIZED_SENTS = []
    G_WORD_TOKENS = []
    G_VOCAB_TABLE = {}
    G_N = n

In [8]:
# run tokenization
initLM()
G_WORD_TOKENS = tokenize(trainData)

In [9]:
print("There are", len(G_WORD_TOKENS), "tokens in the raw training dataset.")

There are 1273123 tokens in the raw training dataset.


Now, with the training dataset tokenized, we can build out vocabulary by creating a frequency table out of the tokens, and convert tokens appeared less than 3 times to `<UNK>`

In [10]:
def oov(tokens):    
    # create a vocab that contains frequency information of each token
    vocab = nltk.FreqDist(tokens)
    # replace tokens with UNK if their freq is too low
    return [token if vocab[token] >= G_FREQ_THRE else G_UNK for token in tokens]
    
G_WORD_TOKENS = oov(G_WORD_TOKENS)
G_VOCAB_TABLE = nltk.FreqDist(G_WORD_TOKENS)

In [11]:
print("The final vocabulary size is", len(G_VOCAB_TABLE.keys()))

demo = list(G_VOCAB_TABLE.items())[:10]
for key, val in demo:
    print(f"{key}: {val}")

The final vocabulary size is 17552
<s>: 50000
facebook: 692
has: 6071
released: 529
a: 31132
report: 312
that: 10032
shows: 289
what: 607
content: 377


**Discussion**

The vocabulary size is `17552` after inserting `SOS/EOS` and replacing less frequent tokens with `UNK`. 

For a ngram model, we're building probability table which contains all combinations of length n given the vocabulary. Therefore, the number of parameters will be $|V|^n$. This is going to be enormous as vocabulary size and `n` increase. Furthermore, the probability table is going to be more and more **sparse** as `n` increases. Hence, we can optimize the probability table by storing only the ngrams that appeared in the training dataset, and provide a default value for unseen data. This can reduce the parameters to store down to `|V|`

### 2.2 $N$-gram Language Modeling

To genralize the process of building n-gram models, we'll create a class that contains all the functions we used in previous section as class methods.

In [31]:
def printDict(dic, n):
    for key, val in dic.items():
        if n == 0:
            break
        print(f"[{key}: {val}]", end=" ")
        n -= 1

class LM:
    # parameters
    _N = 1
    # all individual word tokens
    _word_tokens = []
    # a dict that stores all the frequence of indivicual tokens
    _vocab = {}
    
    _ngrams = []
    # a probability table of all ngrams
    _model = {}
    

    
    def __init__(self, train_sentences, n = 1):
        self._N = max(1, n)
        # tokenize a list of sentences into a single giant list of tokens with SOS/EOS symbols.
        tokens = self._tokenize(train_sentences)
        # update with UNK
        tokens = self._oov(tokens)
        self._word_tokens = tokens
        self._vocab = nltk.FreqDist(self._word_tokens)
    
    def train(self):
        """Create ngram model without doing any smoothing.
        
        This method purely calculates the probability of a given ngram based on
        the frequence of the given ngram and the frequence of its predecessor.
        
        Returns:
            dict: the probability table of ngrams.
        """
        # create grams with window size n
        n_grams = nltk.ngrams(self._word_tokens, self._N)
        self._ngrams = list(n_grams)
        n_vocab = nltk.FreqDist(self._ngrams)
        

        # create grams with window size n - 1
        n_1_grams = nltk.ngrams(self._word_tokens, self._N-1)
        n_1_vocab = nltk.FreqDist(n_1_grams)

        # for unigram
        if self._N == 1:
            tokenSize = len(self._word_tokens)
            self._model = {token: freq / tokenSize for token, freq in n_vocab.items()}
        else:
            # create the probability table for ngram
            self._model = {ngram: freq / n_1_vocab[ngram[:-1]] for ngram, freq in n_vocab.items()}
    
    def getModel(self):
        return self._model
        
            
    def perplexity(self, testData):
        """Calculating perplexity given a list of sentences.
        
        Args:
            testTokens: a list of tokens
        Returns:
            float: the perplexity
        """
        # toknization
        testTokens = self._tokenize(testData)
        return self._perplexity(testTokens)


    def _perplexity(self, testTokens):
        """A private version of calculating perplexity given a list tokens.
        
        Args:
            testTokens: a list of tokens
        Returns:
            float: the perplexity
        """
        # replace oov with UNK
        kownVocab = self._vocab.keys()
        testTokens = [token if token in kownVocab else G_UNK for token in testTokens]
        # get test grams to calculate probabilities
        testGrams = list(nltk.ngrams(testTokens, self._N))
        
        logs = [math.log(self._calcProb(ngram)) for ngram in testGrams]
        return math.exp((-1 / len(testTokens)) * sum(logs))

        
        
    
    def _calcProb(self, ngram):
        """Calculate the probability of a given ngram based on current model.

        For unseen ngrams, the probability is 0.

        Args:
            ngram: the traget.
        Returns:
            float: the probability
        """
        # check if the ngram is known
        return self._model.get(ngram, 0)
    
    def peekTokens(self, n):
        """Have a peek at the tokens
        
        Args:
            n: the number of elements to print.
        """
        for i in range(n):
            print(self._word_tokens[i], end=" ")
    
    def peekVocab(self, n):
        printDict(self._vocab, n)
            
    def getVocabSize(self):
        return len(sel._vocab)
    
    def peekModel(self, n):
        """Have a peek at the ngram table created
        
        Args:
            n: the number of elements to print.
        """
        printDict(self._model, n)
        
    def _tokenize(self, sentences):
        """Transform a list of sentences into a list of tokens.
        
        This function noly only tokenize the input, but also add SOS and EOS symbol before
        and after each sentence.
        
        Args:
            sentences: A list of sentences: str, expected to be the training data.
            
        Returns:
            list: A single giant list of tokens that appeared in the input, with the same order
                  as the input.
        """
        word_tokens = []
        for sentence in sentences:
            # add SOS
            result = [G_SOS] * max(1, (self._N - 1))
            # tokenization
            tokens = self._getCustomWordTokens(sentence)
            # concatenate tokens and EOS
            result.extend(tokens)
            result.append(G_EOS)
            # formulation
            word_tokens.extend(result)
        return word_tokens
    
        
        
    def _oov(self, tokens):
        """This function replace any tokens with low frequence of appearence with UNK
        
        Args:
            tokens: word tokens
        """
        # create a vocab that contains frequnce information of each token
        vocab = nltk.FreqDist(tokens)
        # replae tokens with UNK if their freq is too low
        return [token if vocab[token] >= G_FREQ_THRE else G_UNK for token in tokens]
    
    def _getCustomWordTokens(self, sentence):
        pattern = r'<\w+>|[#$"\(\)\',.!-]|\d+(?:,\d{3})*(?:\.\d+)?|\w+(?:-\w+)*|\S+'
        G_VOCAB_TABLE
        tokenizer = RegexpTokenizer(pattern)
        wordTokens = tokenizer.tokenize(sentence)
        return wordTokens

    def _getNLTKWordTokens(self, sentence):
        wordTokens = word_tokenize(sentence)
        return wordTokens
        
         

By introducing `_calProb(ngram)` function, which checks the probability table for a given ngram and return its corresponding probability or `0` if the ngram is unseen, we can store only ngrams we've seen. So the size of parameters can be shrink down to $|V|$. 

In [32]:
unigram = LM(trainData)
unigram.train()
unigram.peekModel(10)

[('<s>',): 0.039273503031521696] [('facebook',): 0.0005435452819562603] [('has',): 0.0047685887380873645] [('released',): 0.0004155136620734996] [('a',): 0.02445325392754667] [('report',): 0.0002450666589166954] [('that',): 0.007879835648244514] [('shows',): 0.00022700084752219542] [('what',): 0.0004767803268026734] [('content',): 0.0002961222128576736] 

With the model created without using any smoothing technics, let's try to analyze its perplxity againt the training data and dev data.

In [33]:
print("The perplexity of this Unigram model against the training dataset is:", unigram.perplexity(trainData))

The perplexity of this Unigram model against the training dataset is: 796.9548612984283


In [34]:
print("The perplexity of this Unigram against the dev dataset is:", unigram.perplexity(devData))

The perplexity of this Unigram against the dev dataset is: 803.0081866449137


In [17]:
bigram = LM(trainData, 2)
bigram.train()
bigram.peekModel(10)

[('<s>', 'facebook'): 0.00538] [('facebook', 'has'): 0.11271676300578035] [('has', 'released'): 0.015977598418711908] [('released', 'a'): 0.1720226843100189] [('a', 'report'): 0.003950918668893743] [('report', 'that'): 0.041666666666666664] [('that', 'shows'): 0.006379585326953748] [('shows', 'what'): 0.03460207612456748] [('what', 'content'): 0.0016474464579901153] [('content', 'was'): 0.007957559681697613] 

In [35]:
print("The perplexity of this Bigram model against the training dataset is:", bigram.perplexity(trainData))

The perplexity of this Bigram model against the training dataset is: 48.442156606470306


In [36]:
print("The perplexity of this Bigram model against the dev dataset is:", bigram.perplexity(devData))

ValueError: math domain error

**Discussion**

By running the perplexity analysis against the dev dataset, **there's an error occurred** complaining the math domain error. This error demonstrates a problem of our implementation of creating ngrams probability table. **That is, it can't handle unseen data**. When an unseen ngram occurs in the test dataset the probability of it will be `0`. In this implementation, we use log probability during calculating the perplexity to reduce floating number overflow when keep multiplying probabilities. Hence, during calculating the perplexity, we'll have `log(0)` in the equation, which is an undefined behavior. This issue can be resolved by smoothing if we 'donate' some probability mass from seen ngrams to unseen ngrams.

### 2.3 Smoothing

#### 2.3.1 Add-one (Laplace) smoothing

In [20]:
class LM_ADD_K(LM):
    _K = 1
    # model related
    _subVocab = {}
    def __init__(self, sentences, n=1):
        super().__init__(sentences, n)
    
    def train(self, k = 1):
        self._K = k
        self._smoothing()
        
    def _smoothing(self):
        """Create a ngram model with add_k_smoothing.
        
        When k is 1, this function creates model with laplace smoothing.
        
        Agrs:
            k: the increment amount for freq of each ngrams.
        """
        # unigram doesn't support smoothing
        if self._N == 1:
            print("unigram model doesn't support smoothing, continue without smoothing...")
            return super().train()
        
        # create grams with window size n
        n_grams = nltk.ngrams(self._word_tokens, self._N)
        self._ngrams = list(n_grams)
        n_vocab = nltk.FreqDist(self._ngrams)
        
        # create grams with window size n - 1
        n_1_grams = nltk.ngrams(self._word_tokens, self._N-1)
        n_1_vocab = nltk.FreqDist(n_1_grams)
        self._subVocab = n_1_vocab
        
        vocabSize = len(self._vocab)
        self._model = {ngram: (freq + self._K) / (n_1_vocab[ngram[:-1]] + self._K * vocabSize) for ngram, freq in n_vocab.items()}        
    
    
    def _calcProb(self, ngram):
        """Calculate the probability of a given ngram based on current model
        
        When ngram is known, directly return the probability
        When ngram is unkown and smoothing is not used, return 0
        When ngram is unknown but smoothing is used, return smoothed value.
        
        Args:
            ngram: the traget.
        """
        # check if the ngram is known         
        prob = self._model.get(ngram, 0)
        
        if prob == 0:    
            vocabSize = len(self._vocab)
            subFreq = self._subVocab.get(ngram[:-1], 0)
            return self._K / (subFreq + self._K * vocabSize)

        return prob

In [21]:
bigram_laplace = LM_ADD_K(trainData, 2)
bigram_laplace.train()
bigram_laplace.peekModel(10)

[('<s>', 'facebook'): 0.003996920890573188] [('facebook', 'has'): 0.0043301907476430606] [('has', 'released'): 0.004148499343859797] [('released', 'a'): 0.005088214147447597] [('a', 'report'): 0.002547038041245584] [('report', 'that'): 0.0007836990595611285] [('that', 'shows'): 0.0023564385150812066] [('shows', 'what'): 0.000616557367860546] [('what', 'content'): 0.00011013822347045542] [('content', 'was'): 0.00022310223659992193] 

In [22]:
print("The perplexity of this Bigram model with Laplace smoothing against the training dataset is:", bigram_laplace.perplexity(trainData))

The perplexity of this Bigram with Laplace smoothing against the training dataset is: 678.3425847140647


In [23]:
print("The perplexity of this Bigram model with Laplace against the dev dataset is:", bigram_laplace.perplexity(devData))

The perplexity of this Bigram model with Laplace against the dev dataset is: 829.2794789075276


**Discussion**

With smoothing where some probability mass is distributed to all combinations of ngrams including unseen ones, we no longer have the issue discussed in last section. Therefore, we can observe the perplexity of the model against the dev dataset is `829.27`. We also use `_calProb()` technique introduced previously to reduce number of parameters by introducing an additional layer of logic.

However, the perplexity of the new model against the training dataset is much higher than the previous model without smoothing. This case happens because the **training dataset contains only ngrams the model knows**, meaning it expects the probability mass on ngrams in the training dataset to be as much as possible. However, after smoothing, some probability mass is **donated** to unseen ngrams, which is not helping in this case as we'll never encounter unseen ngrams in the training dataset and those mass are sort of **wasted**.

#### 2.3.2: Add-$k$ smoothing

In [25]:
bigram_add_k = LM_ADD_K(trainData, 2)
bigram_add_k.train(0.5)
print("The perplexity of this Bigram model with 0.5 k smoothing against the training dataset is:", bigram_add_k.perplexity(trainData))
print("The perplexity of this Bigram model with 0.5 k smoothing against the dev dataset is:", bigram_add_k.perplexity(devData))

The perplexity of this Bigram model with 0.5 k smoothing against the training dataset is: 459.10639016556837
The perplexity of this Bigram model with 0.5 k smoothing against the dev dataset is: 608.8950847554092


In [26]:
bigram_add_k.train(0.1)
print("The perplexity of this Bigram model with 0.1 k smoothing against the training dataset is:", bigram_add_k.perplexity(trainData))
print("The perplexity of this Bigram model with 0.1 k smoothing against the dev dataset is:", bigram_add_k.perplexity(devData))

The perplexity of this Bigram model with 0.1 k smoothing against the training dataset is: 199.41218240875557
The perplexity of this Bigram model with 0.1 k smoothing against the dev dataset is: 338.2739273653482


In [27]:
bigram_add_k.train(0.001)
print("The perplexity of this Bigram model with 0.001 k smoothing against the training dataset is:", bigram_add_k.perplexity(trainData))
print("The perplexity of this Bigram model with 0.001 k smoothing against the dev dataset is:", bigram_add_k.perplexity(devData))

The perplexity of this Bigram model with 0.001 k smoothing against the training dataset is: 55.751861780817016
The perplexity of this Bigram model with 0.001 k smoothing against the dev dataset is: 217.6393120902775


**Discussion**

Using add-k-smoothing, the perplexity of the model is less than the one with laplace. Especially, when k is getting lower. The reason a lower k generates better result can be the followings. Given a unigram, if the size of its corresponding bigram is limited, after smoothing, the probability of the original seen bigram will be spreaded drastically. For example, For bigrams $a[B]$ (all bigrams starting with letter `a`), if the size of $B$ is 3. For each seen bigram, their probability is $0.333$. After smoothing which spread the probability mass to the additional $|V| - 3$ number of bigrams. Their probability will reduced from $0.333$ to a significant small amount, and the differences between their probability and unseen onces are negligible. Therefore, by lowering the factor `k`, we can reduce the amount of probability mass spread to unseen data, hence remain the dominance of the seen bigrams in the probability mass. On the other hand, observing that keep lowering the factor `k`, the perplexity againt the dev dataset is also reducing, we can also infer that the dev dataset is similar to the training dataset. Otherwise, giving more probability mass to unseen data should produce better result for dev dataset.


#### 2.3.3 Linear Interpolation

In [38]:
class LM_LINEAR_INTERPOLATION(LM):
    _LAMBDAS = []
    _PROB_BI = {}
    _PROB_UNI = {}
    _RAW_DATA = None
    def __init__(self, sentences, n):
        super().__init__(sentences, n)
        self._RAW_DATA = sentences
        self._model = []
        self._LAMBDAS = [1/self._N]*self._N
        
    
    def trainWithUniformWordTokens(self):
        """
        All submodels in this training function will use the same wordTokens which is generated by the highest order model.
        The potential problem is that SOS/EOS are introduced multiple times in the highest order model. EOS/SOS tokens will dominate
        in the lower oder models. 
        Args:
            ls: a list of lambdas in sequential order L0...LN, if not provided, automatically generated with equal weights.
        """
        if self._N == 1:
            print("unigram model doesn't support smoothing, continue without smoothing...")
            return super().train()
        super().__init__(self._RAW_DATA, self._N)
        
        # create grams with window size n
        n_grams = nltk.ngrams(self._word_tokens, self._N)
        self._ngrams = list(n_grams)
        n_vocab = nltk.FreqDist(self._ngrams)
        for i in range(1, len(self._LAMBDAS)):
            # create grams with window size n - i
            n_1_grams = nltk.ngrams(self._word_tokens, self._N-i)
            n_1_vocab = nltk.FreqDist(n_1_grams)
            prob_n_gram = {ngram: (freq) / (n_1_vocab[ngram[:-1]]) for ngram, freq in n_vocab.items()}
            self._model.append(prob_n_gram)
            n_vocab = n_1_vocab
        
        tokenSize = len(self._word_tokens)
        prob_uni_gram =  {token: freq / tokenSize for token, freq in n_vocab.items()}
        self._model.append(prob_uni_gram)
        # each time the probability is appened, so that higher-order models are in the front
        # so reverse the _model
        self._model.reverse()
    
    def train(self):
        """
        Args:
            ls: a list of lambdas in sequential order L0...LN, if not provided, automatically generated with equal weights.
        """
        if self._N == 1:
            print("unigram model doesn't support smoothing, continue without smoothing...")
            return super().train()
        self._model = []
        
        # create grams with window size n
        n_grams = nltk.ngrams(self._word_tokens, self._N)
        self._ngrams = list(n_grams)
        n_vocab = nltk.FreqDist(self._ngrams)
        for i in range(1, self._N + 1):
            print(f"training {i}gram model...")
            subModel = LM(self._RAW_DATA, i)
            subModel.train()
            self._model.append(subModel.getModel())
        
    def optimize(self, data, n_iter, learning_rate):
        """Optimize the lambda parameters using gradient descent.
        
        This method optimize by maximizing the probability of ngrams in the data, instead of minimizing the perpleixty.
        This methods is explained in https://medium.com/mti-technology/n-gram-language-models-b125b9b62e58.
        
        Agrs:
            data: the data we wish to optimize the lambds upon
            n_iter: the number of iterations to perform in finding the optimal lambdas.
            learning_rate: the rate of descent for each iteration.
        Returns:
            list: a list of optimal lambdas for unigram ... ngram.
        """
        # RESET LAMBDAS
        self._LAMBDAS = [1/self._N]*self._N
        
        # toknization
        optTokens = self._tokenize(data)
        # replace oov with UNK
        kownVocab = self._vocab.keys()
        optTokens = [token if token in kownVocab else G_UNK for token in optTokens]
        # get opt grams to calculate probabilities
        optGrams = list(nltk.ngrams(optTokens, self._N))
        
        # get the probability matrix
        probMatrix = self._probMatrix(optGrams)
        
        ngram_probs = probMatrix[:, 1:]
        uniform_prob = probMatrix[:, [0]]
        
        weights = np.array(self._LAMBDAS)
        
        for iteration in range(n_iter):
            # 2. Calculate gradients for each n-gram model
            interpolated_probs = np.sum(probMatrix * weights, axis=1, keepdims=True)
            ngram_gradients = np.mean((ngram_probs - uniform_prob) / interpolated_probs, axis=0)

            # 3. Update interpolation weights for all models
            weights[1:] += learning_rate * ngram_gradients
            weights[0] = 1 - weights[1:].sum()
        return weights
    
    def updateLambdas(self, ls):
        if ls is None:
            return
        # if (sum(ls) != 1):
        #     print("Incorrect lambdas, sum not equal to 1")
        #     return
        if (len(ls) != self._N):
            print("Incorrect lambda length")
            return
        self._LAMBDAS = ls
        
    
    def _probMatrix(self, ngrams):
        # a probability matrix of grams x models
        matrix = np.zeros((len(ngrams), self._N))
        for i in range(len(self._model)):
            matrix[:, [i]] = np.array([self._model[i].get(ngram[self._N - i - 1:], 0) for ngram in ngrams]).reshape(-1, 1)
        print("A glimpse at the probability matrix of the data optimized upon...")
        print(matrix[:5, :5])
        return matrix
        
    def _calcProb(self, ngram):
        prob = 0
        # len(self._model) = self._N
        for i in range(self._N):
            # model stores prob distributions from 0 to N.
            prob += self._model[i].get(ngram[self._N - 1 - i:], 0) * self._LAMBDAS[i]
        if (prob == 0):
            print("calcProb error: not found", ngram)
        return prob
    
    def peekModel(self, n):
        for probs in self._model:
            printDict(probs, n)
            print("")
        

Now, train a trigram model smoothed by linear interpolation with equal weights...

In [29]:
trigram_linear_interpolation = LM_LINEAR_INTERPOLATION(trainData, 3)
trigram_linear_interpolation.train()
print("Have a look at the submodels:")
trigram_linear_interpolation.peekModel(3)

training 1gram model...
training 2gram model...
training 3gram model...
Have a look at the submodels:
[('<s>',): 0.039273503031521696] [('facebook',): 0.0005435452819562603] [('has',): 0.0047685887380873645] 
[('<s>', 'facebook'): 0.00538] [('facebook', 'has'): 0.11271676300578035] [('has', 'released'): 0.015977598418711908] 
[('<s>', '<s>', 'facebook'): 0.00538] [('<s>', 'facebook', 'has'): 0.24907063197026022] [('facebook', 'has', 'released'): 0.02564102564102564] 


In [30]:
print("The perplexity of this trigram model with linear interpolation against the training dataset is:", trigram_linear_interpolation.perplexity(trainData))
print("The perplexity of this trigram model with linear interpolation against the dev dataset is:", trigram_linear_interpolation.perplexity(devData))

The perplexity of this trigram model with linear interpolation against the training dataset is: 12.953025318001819
The perplexity of this trigram model with linear interpolation against the dev dataset is: 123.6382902418981


Now, try manually to optimize the lambdas to perform best on dev dataset

In [63]:
trigram_linear_interpolation.updateLambdas([0.3, 0.2, 0.5])
print("The perplexity of this model against the dev dataset is:", trigram_linear_interpolation.perplexity(devData))

The perplexity of this model against the dev dataset is: 132.4205743334757


In [64]:
trigram_linear_interpolation.updateLambdas([0.2, 0.4, 0.4])
print("The perplexity of this model against the dev dataset is:", trigram_linear_interpolation.perplexity(devData))

The perplexity of this model against the dev dataset is: 122.83927845909868


In [65]:
trigram_linear_interpolation.updateLambdas([0.2, 0.7, 0.1])
print("The perplexity of this model against the dev dataset is:", trigram_linear_interpolation.perplexity(devData))

The perplexity of this model against the dev dataset is: 127.17716778785392


With 3 trials, we decide to use $\lambda_1 = 0.2, \lambda_2 = 0.4, \lambda_3 = 0.4$ for our manually optimized weights to test.

In [66]:
trigram_linear_interpolation.updateLambdas([0.2, 0.4, 0.4])
print("The perplexity of this model against the test dataset is:", trigram_linear_interpolation.perplexity(testData))

The perplexity of this model against the test dataset is: 121.94002570853857


**Discussion**

After using linear interpolation, we can find the perplexity drastically reduced compared to add-k smoothing. When training a ngram model, we always wish to increase `n` as it will give us more history information in predicting. However, as `n` increases, the model will be hungry for data, and without enough data, the probability table of model will be sparse, leading worse performance than lower order models. Linear interpolation gives us a flexibility to take context advantages of higher order model and mitigate its sparsity problem with lower order models. 

##### **Optimization**:

Now, optimize the lambdas using [Gradient Descent](https://medium.com/mti-technology/n-gram-language-models-b125b9b62e58). Gradient Descent is normally used for unconstrained optimization problems, and this time, we have a constraint that the sum of the weights should be `1`. To mitigate this, we'll set the weight of the unigram to be $1 - \lambda_2 - \lambda_3$. Then, the formula of the probability can be writen as:

$$
\begin{align}
P &= \frac{1}{N_{word}} \sum_{word} log P(word) \\
  &= \frac{1}{N_{word}} \sum_{word} log ((1 - \lambda_2 - \lambda_3)P(w) + (\lambda_2 P(w | w_{-1})) + (\lambda_3 P(w | w_{-1}w_{-2})))
\end{align}
$$

To differentiate upon each lambda, we can have the formula for $\lambda_j$ as the followings:

$$
\begin{align}
\frac{\delta P}{\delta \lambda_j} &= \frac{1}{N_{word}} \sum_{word} \frac{1}{P(word)} \frac{\delta P(word)}{\delta \lambda_j} \\
&= \frac{1}{N_{word}} \sum_{word} \frac{1}{P(word)}(-P(w) + P(w | w_{\dots}w_{j-1}) )
\end{align}
$$


In [67]:
trigram_linear_interpolation.optimize(devData, 800, 0.001)

A glimpse at the probability matrix of the data optimized upon...
[[1.72803413e-04 4.32000000e-03 4.32000000e-03]
 [4.76858874e-03 3.18181818e-02 3.24074074e-02]
 [4.86991438e-05 1.15302257e-03 0.00000000e+00]
 [2.44532539e-02 3.22580645e-02 0.00000000e+00]
 [2.12469651e-02 5.23577027e-02 0.00000000e+00]]


array([0.24704623, 0.44271949, 0.31023428])

Update the model with the optimal lambdas and we expect the perplexity againt the dev datset to be smaller.

In [49]:
trigram_linear_interpolation.updateLambdas([0.24704623, 0.44271949, 0.31023428])
print("The perplexity of this model against the training dataset is:", trigram_linear_interpolation.perplexity(trainData))
print("The perplexity of this model against the dev dataset is:", trigram_linear_interpolation.perplexity(devData))
print("The perplexity of this model against the test dataset is:", trigram_linear_interpolation.perplexity(testData))

The perplexity of this model against the training dataset is: 12.898964375272874
The perplexity of this model against the dev dataset is: 170.5045386731055
The perplexity of this model against the test dataset is: 119.75103728465692


## 3 Preposition Prediction

In [58]:
!wget -O dev.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.out

!wget -O test.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/test.in



--2023-10-23 01:15:48--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210427 (205K) [text/plain]
Saving to: 'dev.in'

dev.in              100%[===================>] 205.50K  --.-KB/s    in 0.1s    

2023-10-23 01:15:49 (1.51 MB/s) - 'dev.in' saved [210427/210427]

--2023-10-23 01:15:49--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.out
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10

In [77]:
import random
import re
import math

In [59]:
with open("dev.in", 'r') as rawFile:
    devData = [line.strip() for line in rawFile]
    
with open("dev.out", 'r') as ansFile:
    devAns = [line.strip() for line in ansFile]

In [114]:
G_PREP = "<PREP>"


class PREP_PREDICTOR(LM_LINEAR_INTERPOLATION):
    _ALL_PREP_TOKENS = ["for", "in", "of", "at", "on"]
    
    def __init__(self, trainData, n):
        super().__init__(trainData, n)
        
        
    def _preprocess(self, rawData, ansData):
        """preprocess datainput by combining the trainData and the ansData
        Args:
            rawData: the training dataset with <PREP> masked
            ansData: the answers in corresponding masks for each trainData
        Returns:
            list: a list of data combined with answers
        """
        combinedData = []
        n = len(rawData)
        for i in range(n):
            data = rawData[i]
            ansTokens = ansData[i].split(" ")
            for ans in ansTokens:
                data = data.replace(G_PREP, ans, 1)
            combinedData.append(data)
        return combinedData
    
    def perplexity(self, testData, testAns):
        combinedData, _ = self._preprocess(testData, testAns)
        return super().perplexity(combinedData)
    
    def optimize(self, data, ans, n_iter, learning_rate):
        combinedData = self._preprocess(data, ans)
        return super().optimize(combinedData, n_iter, learning_rate)
    
    def predictOne(self, sentTokens):
        """given a list of tokens, predict all preps

        Args:
            data: string sentence.
        """
        resultPrep = []
        # startidx for searching <PREP> pattern
        startIdx = 0
        while(1):
            # print("---------")
            try:
                index = sentTokens.index(G_PREP, startIdx)
                startIdx = index + 1
                # n-1gram without the GPREP 
                b4_n_1grams = sentTokens[index - self._N + 1: index]
                # 2 grams after GPREP
                af_2grams = sentTokens[index+1: index+3]
                # find the best prep and the associated perplexity
                prepToken, _ = self._bestPREP(b4_n_1grams, af_2grams)
                sentTokens[index] = prepToken
                resultPrep.append(prepToken)
            except ValueError:
                break
        return resultPrep
    
    def predictData(self, data, path):
        """Predict all preps in a dataset of sentences, and write result to a file.
        Args:
            data: a list of sentences containing <PREP> masks.
            path: the path of the file to store the result
        """
        with open(path, 'w') as file:
            for sentence in data:
                sentTokens = self._tokenize([sentence])
                predicts = self.predictOne(sentTokens)
                self._writeListToFile(predicts, file)
    
    def performacePerplexity(self, data, ans):
        """Evaluate the performace on a dataset
        Args:
            data: a list of sentences masked by <PREP> to be predicted
            ans: a list of corresponding answers
        Return: rate of correctness
        """
        correctCount = 0
        totalCount = 0
        for i in range(len(data)):
            sentTokens = self._tokenize([data[i]])
            predicts = self.predictOne(sentTokens)
            actuals = ans[i].split(" ")
            for a, b in zip(predicts, actuals):
                totalCount += 1
                if (a == b):
                    correctCount += 1
        return correctCount / totalCount
            
    
    def _writeListToFile(self, lst, file):
        """Helper function to write a list to a file
        """
        file.write(lst[0])
        for item in lst[1:]:
            file.write(" " + item)
        file.write("\n")
        
    
    def _bestPREP(self, b4, af_2):
        """Given a n- tokens and 2 tokens after the target, return the best Prep with lowest perplexity.

        This function concatenate the all previous grams relative to the N, and the following 2 grams, and compare the perplexity
        
        Agrs:
            b4: n-1grams before the target
            af_2: 2 grams after the target
        """
        bestPrep = None
        bestPPL = math.inf
        for prepToken in self._ALL_PREP_TOKENS:
            candidateTokens = b4 + [prepToken] + af_2
            ppl = self._perplexity(candidateTokens)
            if ppl < bestPPL:
                bestPPL = ppl
                bestPrep = prepToken
        return bestPrep, bestPPL

In [115]:
predictor = PREP_PREDICTOR(trainData, 3)
predictor.train()
print("Have a look at the submodels:")
predictor.peekModel(2)

training 1gram model...
training 2gram model...
training 3gram model...
Have a look at the submodels:
[('<s>',): 0.039273503031521696] [('facebook',): 0.0005435452819562603] 
[('<s>', 'facebook'): 0.00538] [('facebook', 'has'): 0.11271676300578035] 
[('<s>', '<s>', 'facebook'): 0.00538] [('<s>', 'facebook', 'has'): 0.24907063197026022] 


In [93]:
predictor.optimize(devData, devAns, 1000, 0.001)

A glimpse at the probability matrix of the data optimized upon...
[[2.16789737e-04 5.60000000e-04 5.60000000e-04]
 [3.77025629e-04 6.52173913e-02 2.50000000e-01]
 [6.04811947e-05 7.08333333e-02 8.88888889e-01]
 [6.28376049e-05 1.00000000e+00 1.00000000e+00]
 [1.04286860e-02 3.75000000e-02 3.89610390e-02]]


array([0.24890615, 0.45547281, 0.29562105])

In [108]:
predictor.updateLambdas([0.24890615, 0.45547281, 0.29562105])

In this predictor, it predicts the preposition by selecting the one that generates the lowest perplexity in its neighborhood. Unlike traditional prediction that treats the PREP as the next gram and select the prep with the highest probability, in this approach, we concatenate all n-1grams, the candidate prep and the **following 2 grams**, and select the n+2grams with the lowest perplexity (the same as selecting the n+2gram with highest probability).

In [117]:
print("The accuracy of predicting the devData is", predictor.performace(devData, devAns))

The accuracy of predicting the devData is 0.7131837307152875


In [118]:
# generate test result
with open("test.in", 'r') as testFile:
    testData = [line.strip() for line in testFile]

predictor.predictData(testData, "test.out")